In [18]:
from preprocessing.video import Video
from preprocessing.dto.sequence_dto import SequenceDto
from preprocessing.inference_service.external_model.yolov7_pose_tracker.utils.plots import plot_one_box_kpt, \
    plot_skeleton_kpts
from pathlib import Path
import cv2
import numpy as np

v = Video(Path('/home/rafa/SVW/Videos/soccer/252___64eaaaffa5284ed8aae4682f4b08e9ba.mp4'))

sequences = Video._expand_sequences_df(v.build_sequences())
sequences.head(100)

,video,frame_id,bbox,max_wh,skeleton,...,frame_copied_from,start_frame,class,stem,sequence_id
0,/home/rafa/SVW/Videos/soccer/252___64eaaaffa52...,2,"[558.0, 443.0, 281.0, 378.0]","[351, 441]","[649.206, 482.71634, 0.021119986, 647.449, 478...",...,2,2,soccer,252___64eaaaffa5284ed8aae4682f4b08e9ba.mp4,0
1,/home/rafa/SVW/Videos/soccer/252___64eaaaffa52...,3,"[546.0, 444.0, 294.0, 377.0]","[351, 441]","[635.2714, 482.8635, 0.03939121, 632.0901, 476...",...,3,2,soccer,252___64eaaaffa5284ed8aae4682f4b08e9ba.mp4,0
2,/home/rafa/SVW/Videos/soccer/252___64eaaaffa52...,4,"[532.0, 443.0, 306.0, 379.0]","[351, 441]","[623.4775, 482.0298, 0.054746985, 620.0984, 47...",...,4,2,soccer,252___64eaaaffa5284ed8aae4682f4b08e9ba.mp4,0
3,/home/rafa/SVW/Videos/soccer/252___64eaaaffa52...,5,"[516.0, 439.0, 314.0, 385.0]","[351, 441]","[615.47217, 479.28595, 0.015253341, 613.51685,...",...,5,2,soccer,252___64eaaaffa5284ed8aae4682f4b08e9ba.mp4,0
4,/home/rafa/SVW/Videos/soccer/252___64eaaaffa52...,6,"[502.0, 435.0, 314.0, 389.0]","[351, 441]","[609.1815, 475.7215, 0.012488553, 606.49, 469....",...,6,2,soccer,252___64eaaaffa5284ed8aae4682f4b08e9ba.mp4,0
...,...,...,...,...,...,...,...,...,...,...,...
31,/home/rafa/SVW/Videos/soccer/252___64eaaaffa52...,33,"[282.0, 459.0, 133.0, 252.0]","[149, 254]","[335.61484, 486.3452, 0.020991134, 332.6997, 4...",...,33,2,soccer,252___64eaaaffa5284ed8aae4682f4b08e9ba.mp4,2
0,/home/rafa/SVW/Videos/soccer/252___64eaaaffa52...,34,"[0.0, 440.0, 320.0, 423.0]","[321, 460]","[150.01457, 483.5056, 0.076504484, 151.99501, ...",...,34,34,soccer,252___64eaaaffa5284ed8aae4682f4b08e9ba.mp4,3
1,/home/rafa/SVW/Videos/soccer/252___64eaaaffa52...,35,"[0.0, 439.0, 311.0, 435.0]","[321, 460]","[131.82893, 483.80646, 0.104149505, 134.90866,...",...,35,34,soccer,252___64eaaaffa5284ed8aae4682f4b08e9ba.mp4,3
2,/home/rafa/SVW/Videos/soccer/252___64eaaaffa52...,36,"[0.0, 440.0, 316.0, 430.0]","[321, 460]","[142.99336, 487.13467, 0.014704234, 139.18492,...",...,36,34,soccer,252___64eaaaffa5284ed8aae4682f4b08e9ba.mp4,3


In [19]:
cap = cv2.VideoCapture(v.video_reshaped_path.as_posix())
video = [np.zeros((1080, 1920, 3))]
while (cap.isOpened()):
    ret, img = cap.read()
    if not ret:
        break
    video.append(img)
cap.release()



In [20]:

from preprocessing.dto.detection_dto import DetectionDto
from random import randint
from utils.color_generator import ColorGenerator

cg = ColorGenerator()

frames = sequences['frame_copied_from'].unique()
for frame in frames:
    # frame = np.zeros
    frame_cpy = np.copy(video[frame])
    df = sequences[sequences['frame_copied_from'] == frame]
    skeletons =df[SequenceDto.SKELETON].to_list()
    bboxes =df[SequenceDto.BBOX]
    seq_ids = df['sequence_id']
    i = 0
    for bbox, skeleton, pid in zip(bboxes, skeletons, seq_ids):
        accs = np.average(skeleton[2::3])
        if accs < 0.5:
            continue
        x = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]]
        angles = np.array(sequences[sequences['sequence_id'] == pid][SequenceDto.SKELETON_ANGLES].to_list())
        if np.sum((angles.max(axis=0) - angles.min(axis=0))>  0.25) < 3:
            continue
        delta = np.max(angles.max(axis=0) - angles.min(axis=0))
        # if delta < 0.3:
        #     continue
        color = cg.id_to_random_color(pid)
        plot_one_box_kpt(x=x, im=frame_cpy, kpt_label=True, kpts=skeleton, steps=3, color=color, line_thickness=8)
        cv2.putText(frame_cpy, f"{delta}", (int(bbox[0]), int(bbox[1])), cv2.FONT_HERSHEY_SIMPLEX, 1,color , 3)
        i+=1
        # print(skeleton.reshape((-1, 3)))
    cv2.imshow('frame', frame_cpy)
    cv2.waitKey(0)
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x8ccd160) is not the object's thread (0x8fb5160).
Cannot move to target thread (0x8ccd160)

QObject::moveToThread: Current thread (0x8ccd160) is not the object's thread (0x8fb5160).
Cannot move to target thread (0x8ccd160)

QObject::moveToThread: Current thread (0x8ccd160) is not the object's thread (0x8fb5160).
Cannot move to target thread (0x8ccd160)

QObject::moveToThread: Current thread (0x8ccd160) is not the object's thread (0x8fb5160).
Cannot move to target thread (0x8ccd160)

QObject::moveToThread: Current thread (0x8ccd160) is not the object's thread (0x8fb5160).
Cannot move to target thread (0x8ccd160)

QObject::moveToThread: Current thread (0x8ccd160) is not the object's thread (0x8fb5160).
Cannot move to target thread (0x8ccd160)

QObject::moveToThread: Current thread (0x8ccd160) is not the object's thread (0x8fb5160).
Cannot move to target thread (0x8ccd160)

QObject::moveToThread: Current thread (0x8ccd160) is not the object's thread